

*   In this mini project 3, I use "zaanind/sinhala_englsih_parrel_corpus" data set in hugging face web site. link:-https://huggingface.co/datasets/zaanind/sinhala_englsih_parrel_corpus .


*   And I use thilina/mt5-sinhalese-english model in hugging face web site, link:- https://huggingface.co/thilina/mt5-sinhalese-english .









In [ ]:
!nvidia-smi

Sun Apr 21 07:28:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.6 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM


In [ ]:
model_checkpoint = "thilina/mt5-sinhalese-english"

In [ ]:
raw_datasets = load_dataset("zaanind/sinhala_englsih_parrel_corpus")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/80684 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['english', 'sinhala'],
        num_rows: 80684
    })
})

In [ ]:

raw_datasets['train'][1]

{'english': 'you will receive a confirmation code after completing the registration',
 'sinhala': 'ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි"]))

{'input_ids': [[50043, 146489, 12430, 115838, 28433, 75079, 8242, 68077, 64845, 1489, 259, 45820, 61543, 3165, 76171, 5872, 4887, 2177, 22846, 9071, 1851, 3085, 147217, 5005, 8892, 42963, 10431, 7299, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
input_text = "Hello,I am saman"
tokenized_input = tokenizer(input_text, return_tensors="tf")


In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


tf_model.h5:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at thilina/mt5-sinhalese-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [ ]:
translated = model.generate(tokenized_input["input_ids"])


/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
print("Translated Text:", translated_text)


Translated Text: හෙලෝ, මම සාමන්


In [ ]:
def translate_to_sinhala(text):
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_ids = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

# Step 5: Test the translation function
english_text = "Now I am going to go "
sinhala_translation = translate_to_sinhala(english_text)
print(f"English: {english_text}")
print(f"Sinhala: {sinhala_translation}")

English: Now I am going to go 
Sinhala: දැන් මම යන්නම්.


In [ ]:
def translate_to_sinhala(text):
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_ids = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

# Step 5: Test the translation function
english_text = "University of kelaniya is a very good university in sri lanka for high education "
sinhala_translation = translate_to_sinhala(english_text)
print(f"English: {english_text}")
print(f"Sinhala: {sinhala_translation}")

English: University of kelaniya is a very good university in sri lanka for high education 
Sinhala: කෙලෙයානියා විශ්වවිද් යාලයේ ඉතා හොඳ අධ් යාපනයක් ලබා


In [ ]:
def translate_to_sinhala(text):
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_ids = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

# Step 5: Test the translation function
english_text = " By using internet we can get more thins to our mind "
sinhala_translation = translate_to_sinhala(english_text)
print(f"English: {english_text}")
print(f"Sinhala: {sinhala_translation}")

English:  By using internet we can get more thins to our mind 
Sinhala: අන්තර්ජාලය භාවිතයෙන් අපේ මනසට තව ටිකක් ඈතට


In [ ]:
def translate_to_sinhala(text):
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_ids = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

# Step 5: Test the translation function
english_text = "I am reading books "
sinhala_translation = translate_to_sinhala(english_text)
print(f"English: {english_text}")
print(f"Sinhala: {sinhala_translation}")

English: I am reading books 
Sinhala: මම පොත් කියවනවා


In [28]:
def translate_to_sinhala(text):
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_ids = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

# Step 5: Test the translation function
english_text = "I like to read books not only that  my mother always like   "
sinhala_translation = translate_to_sinhala(english_text)
print(f"English: {english_text}")
print(f"Sinhala: {sinhala_translation}")

English: I like to read books not only that  my mother always like   
Sinhala: මම පොත් කියවන්න කැමතියි මගේ අම්මා කැමති විදියට.


In [33]:
def translate_to_sinhala(text):
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_ids = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

# Step 5: Test the translation function
english_text = "I like Computer Science"
sinhala_translation = translate_to_sinhala(english_text)
print(f"English: {english_text}")
print(f"Sinhala: {sinhala_translation}")

English: I like Computer Science
Sinhala: මම කැමතියි පරිගණක විද් යාව


In [34]:
def translate_to_sinhala(text):
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_ids = model.generate(**tokenized_text)
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

# Step 5: Test the translation function
english_text = "water"
sinhala_translation = translate_to_sinhala(english_text)
print(f"English: {english_text}")
print(f"Sinhala: {sinhala_translation}")

English: water
Sinhala: වතුර
